# Request and Download OOI Data

**Purpose:** This notebook creates requests for data and QARTOD QC test results that are available from OOINet and from the OOI dev1 server. QC tests associated with datasets from OOINet have already been implemented in production by the Data Team. The dev1 server is where datasets with results of QARTOD tests in development are hosted. Access to dev1 is restricted to OOI personnel on the internal network.

The requests built below include the retrieval method, data stream, and either the reference designator or site, node, and sensor combination for a specific instrument to request data through the OOI M2M API. The requested datasets can also be limited to a time period defined by start datetime and end datetime parameters.

After downloading the datasets and performing preprocessing to prepare the data for analysis, the datasets are saved locally to an external data folder for the next step in testing and analyzing QARTOD test results.

### Import modules used in this notebook

In [1]:
# Import libraries available from main conda channels or conda-forge
import xarray as xr
import os
import warnings
warnings.filterwarnings("ignore")

# Import "common" module from ooi-data-explorations library
from ooi_data_explorations import common

# Import qartod_testing project modules
import qartod_testing.data_processing as dp

### QARTOD in Production: Request data from the OOINet THREDDS catalog

The next 4 subsections are different attempts a requesting data from OOINet. 
Downloading data with `M2M.download_netCDF_files()` was successful for a couple of PHSEN instruments, although it does not do any preprocessing before saving these datasets. For other datasets I usually run into a file or directory not found error at the local directory where I am trying to write data.

##### Define data parameters

In [2]:
# Setup parameters needed to request data
refdes = "GA01SUMO-RID16-05-PCO2WB000"              # Coastal Pioneer Array (NES) - Central Surface Mooring CTD Bottom-pumped, is this the same as site, node, sensor?
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "pco2w_abc_instrument"          # name of data stream

# Site, node, and sensor info from deconstructed reference designator
[site, node, sensor] = refdes.split('-', 2)

##### Download data with OOINet M2M module and xarray 

In [3]:
# Routine in data_processing module from this project to download the gold copy THREDDs datasets
# Variable 'files' contains list of catalog URLs for downloaded datasets 

files = dp.ooinet_gold_copy_request(refdes, method, stream)

##### Using ooi_data_explorations modules

In [5]:
# Load data with common module
data = common.load_gc_thredds(site,node,sensor,method,stream)    # Request the gold copy data through THREDDs catalog

# I couldn't use dask with PCO2W data request since some of the data was an object data type, which is not supported for auto-chunking.

# It looks like the OOINet module method attempts to avoid collecting ancillary files in addition to the requested sensor files which could add time to the download and open dataset step.
# load_gc_thredds() also calls process_file() within gc_collect() so we achieve the same preprocessing as in the preprocess() defined above.
# Still not certain why preproccessing with ooi_data_explorations library returns a different dataset from the OOINet preprocessing routine.

In [6]:
# Make a copy of the data with a unique name
ds_prod = data.copy()
ds_prod

<xarray.Dataset>
Dimensions:                                   (time: 11452, spectrum: 14)
Coordinates:
  * spectrum                                  (spectrum) int32 0 1 2 ... 12 13
  * time                                      (time) datetime64[ns] 2015-03-1...
Data variables: (12/24)
    pco2w_thermistor_temperature_qc_executed  (time) uint8 1 1 1 1 1 ... 1 1 1 1
    pco2_seawater_qartod_results              (time) uint8 9 9 9 9 9 ... 1 1 1 1
    pco2_seawater_qc_executed                 (time) uint8 29 29 29 ... 29 29 29
    absorbance_ratio_434_qc_executed          (time) uint8 9 9 9 9 9 ... 9 9 9 9
    absorbance_blank_620                      (time) float64 3.377e+03 ... 2....
    record_type                               (time) uint8 4 4 4 4 4 ... 4 4 4 4
    ...                                        ...
    pco2w_thermistor_temperature              (time) float64 13.61 ... 15.97
    absorbance_ratio_620_qc_executed          (time) uint8 9 9 9 9 9 ... 9 9 9 9
    record_time                               (time) uint32 3509301909 ... 35...
    absorbance_ratio_620_qc_results           (time) uint8 9 9 9 9 9 ... 9 9 9 9
    thermistor_raw                            (time) int16 1962 1964 ... 1859
    pco2_seawater                             (time) float64 -1e+07 ... 498.9
Attributes: (12/69)
    node:                               RID16
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             pco2w_abc_instrument
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                -42.92283
    lon:                                -42.4393

In [ ]:
# Save downloaded data locally
prod_path = dp.build_data_path(refdes, method, stream, prefix='prod', folder='external') # added this folder='external' as I was updating the notebook. not sure if I want to put requested data in interim or external going forward

ds_prod.to_netcdf(path=prod_path)

### QARTOD in Development: Request data from dev1 server

We may also want to examine new QARTOD tests which are on staging in the Dev-1 environment before they are moved to production. The Development environemt at ooinet-dev1-west.intra.oceanobservatories.org. In order to access data on Dev-1, you need to be granted access and be connected to the CI-West VPN (vpn-west.oceanobservatories.org) at Oregon State.

In [2]:
# Setup parameters needed to request data 
# Check that instrument parameters match an available OOI dataset on dev1 server
# Maybe change this section to look for data sets programatically with ooi-data-explorations functions (list platforms/sites, list methods, list streams,...)

refdes = "CP03ISSM-RID27-03-CTDBPC000"              # Coastal Pioneer Array (NES) - Inshore Surface Mooring Near Surface Instrument Frame - Bottom-pumped CTD
method = "recovered_inst"                           # non-decimated data from recovered instrument
stream = "ctdbp_cdef_instrument_recovered"          # name of data stream

# Site, node, and sensor info from deconstructed reference designator
[site, node, sensor] = refdes.split('-', 2)

In [3]:
# Set optional parameters 
# We specify a date range to control the size of the dataset requested, usually focusing on one deployment 
params = {
  'beginDT':'2019-09-27T00:00:00.000Z',
  'endDT':'2020-10-31T00:00:00.000Z',
  'format':'application/netcdf',
  'include_provenance':'true',
  'include_annotations':'true'
}

We are using a different process for downloading data than in the QARTOD in production section since the default M2M URLs that are set within the M2M module functions connect to OOINet rather than Dev-1. 
The development environment also doesn't have a "gold copy" equivalent THREDDs catalog. Instead we'll have to do the normal request and wait for the datasets to be assembled and made available for download. Different functions to request non-gold copy datasets from OOINet exist in the OOINet and ooi-data-explorations modules.


Both the data catalog and file server URLs are created by swapping out "ooinet" in the M2M URLs for the Dev-1 server name. 
Overall, the data request process through this section is modeled after this M2M example: https://github.com/ooi-data-review/2018-data-workshops/blob/master/chemistry/examples/quickstart_python.ipynb 

In [4]:
# Connect to ci-west vpn before running this cell to download data
data = dp.dev1_data_request(site, node, sensor, method, stream, params)

Waiting for Dev 01 to process and prepare data request, this may take up to 20 minutes.
Waiting: 100%|██████████| 400/400 [01:00<00:00,  6.64it/s]


In [5]:
# Make a copy of the data with a unique name
ds_dev = data.copy()
ds_dev

<xarray.Dataset>
Dimensions:                                            (time: 732380)
Coordinates:
  * time                                               (time) datetime64[ns] ...
Data variables: (12/35)
    sea_water_pressure_qc_results                      (time) uint8 dask.array<chunksize=(2,), meta=np.ndarray>
    sea_water_pressure                                 (time) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_results   (time) uint8 dask.array<chunksize=(2,), meta=np.ndarray>
    sea_water_pressure_qc_executed                     (time) uint8 dask.array<chunksize=(2,), meta=np.ndarray>
    ctd_time                                           (time) int32 dask.array<chunksize=(2,), meta=np.ndarray>
    sea_water_practical_salinity_qc_executed           (time) uint8 dask.array<chunksize=(2,), meta=np.ndarray>
    ...                                                 ...
    sea_water_density                                  (time) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    depth                                              (time) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    sea_water_pressure_qartod_results                  (time) uint8 dask.array<chunksize=(2,), meta=np.ndarray>
    sea_water_electrical_conductivity_qartod_executed  (time) object dask.array<chunksize=(2,), meta=np.ndarray>
    sea_water_temperature                              (time) float64 dask.array<chunksize=(2,), meta=np.ndarray>
    sea_water_practical_salinity_qartod_executed       (time) object dask.array<chunksize=(2,), meta=np.ndarray>
Attributes: (12/69)
    node:                               RID27
    comment:                            Data collected from the OOI Dev01 M2M...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  recovered_inst
    stream:                             ctdbp_cdef_instrument_recovered
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                40.3594
    lon:                                -70.8851

##### Save datasets for test in development to external data folder for further processing

In [7]:
# path to dev subfolder in external data folder from notebook folder
dev_folder = os.path.relpath('../data/external/dev')

# Make the dev folder if it does not already exist
os.makedirs(dev_folder, exist_ok=True)

# build ds_dev filename from dataset attributes
dev_filename = '-'.join(('dev',ds_dev.id,))+'.nc'     

# build full relative path with ds_dev filename
dev_path=os.path.join(dev_folder, dev_filename)           

# provide both relative path and filename for ds_dev in path parameter
# currently, the resulting dataset could be multiple files and the file name from the file server is not reflected in the file name used here (so no deployment in file name)
ds_dev.to_netcdf(path=dev_path)                          